#Face Mask Detection Model

In [ ]:
%pip install kagglehub

In [76]:
# นำเข้า modules ของ libraries
import os
import re
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xml.etree.ElementTree as ET
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from kagglehub import login, dataset_download

In [2]:
# Login เข้า Kaggle (ต้องดาวโหลด์ใบ credentail จาก kaggle ก่อนแล้วเอา username กับ key มากรอก)
# หรือถ้าไม่มีให้ใช้ { "username":"warinsaipanya", "key":"e9d6ec16f057b50fac90908de9990b27" } dict นี้
login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# ดาวโหลด์ตัว datasets จาก kaggle เก็บไว้ใน path ที่ระบุ
path = dataset_download("andrewmvd/face-mask-detection")
print(f"path ของไฟล์ dataset อยู่ที่: {path}")

100%|██████████| 398M/398M [00:07<00:00, 59.4MB/s]

Extracting files...


path ของไฟล์ dataset อยู่ที่: /root/.cache/kagglehub/datasets/andrewmvd/face-mask-detection/versions/1


In [64]:
# classes เป็น list ไว้เก็บคำตอบ
classes = ["mask worn incorrectly", "with mask", "without mask"]

In [65]:
# path ที่เก็บข้อมูลรูปภาพไว้ train
folder_path = f"{path}/images"
# path ที่เก็บข้อมูลไฟล์ xml
xml_folder = f"{path}/annotations"

In [66]:
# สร้าง list ไว้เก็บข้อมูล x และ y
# x สำหรับเก็บข้อมูล features ของรูปภาพ
x = []
# y สำหรับเก็บคำตอบของรูปภาพ
y = []

In [67]:
# function สำหรับการอ่านข้อมูลในไฟล์ xml
def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # เก็บข้อมูลใน tag ของ object
    objects = []

    # วน loop หา element ที่มีชื่อว่า object และได้กลับคืนมาเป็น list แล้ววน loop
    for obj in root.findall("object"):
        # หา tag ที่ชื่อ name แล้วอ่าน content ด้านในเก็บ class คำตอบของรูป
        label = obj.find("name").text
        # หา tag bndbox
        bndbox = obj.find("bndbox")
        # อ่านข้อมูลด้านใน element bndbox แล้วแปลงค่าเป็น int
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        # เพื่อ dictionary เข้าไปใน list
        objects.append({"label": label, "bbox": [xmin, ymin, xmax, ymax]})

    return objects

In [68]:
def extract_number(filename):
    match = re.search(r"\d+", filename)
    return int(match.group()) if match else -1

In [69]:
# list เก็บชื่อไฟล์ xml แบบเรียงลำดับชื่อไฟล์เป็นไฟล์ข้อมูลสำหรับแต่หล่ะรูปภาพ
xml_files = sorted(os.listdir(xml_folder), key=extract_number)
# สร้าง object ของ label encoder ไว้ normalize ตัว label ของรูป
encoder = LabelEncoder()

# แสดงผล list ที่เก็บไฟล์ xml ไว้
print(xml_files)

['maksssksksss0.xml', 'maksssksksss1.xml', 'maksssksksss2.xml', 'maksssksksss3.xml', 'maksssksksss4.xml', 'maksssksksss5.xml', 'maksssksksss6.xml', 'maksssksksss7.xml', 'maksssksksss8.xml', 'maksssksksss9.xml', 'maksssksksss10.xml', 'maksssksksss11.xml', 'maksssksksss12.xml', 'maksssksksss13.xml', 'maksssksksss14.xml', 'maksssksksss15.xml', 'maksssksksss16.xml', 'maksssksksss17.xml', 'maksssksksss18.xml', 'maksssksksss19.xml', 'maksssksksss20.xml', 'maksssksksss21.xml', 'maksssksksss22.xml', 'maksssksksss23.xml', 'maksssksksss24.xml', 'maksssksksss25.xml', 'maksssksksss26.xml', 'maksssksksss27.xml', 'maksssksksss28.xml', 'maksssksksss29.xml', 'maksssksksss30.xml', 'maksssksksss31.xml', 'maksssksksss32.xml', 'maksssksksss33.xml', 'maksssksksss34.xml', 'maksssksksss35.xml', 'maksssksksss36.xml', 'maksssksksss37.xml', 'maksssksksss38.xml', 'maksssksksss39.xml', 'maksssksksss40.xml', 'maksssksksss41.xml', 'maksssksksss42.xml', 'maksssksksss43.xml', 'maksssksksss44.xml', 'maksssksksss45.xml

In [70]:
# วน loop แต่ล่ะชื่อ xml file
for file in xml_files:
    # เอา path ของที่เก็บไฟล์ xml  มา join กับชื่อไฟล์ จะได้ตำแหน่ง path ที่แท้จริงของไฟล์ xml
    file_path = os.path.join(xml_folder, file)

    # เช็คตรวจสอบไฟล์ว่ามี path ไฟล์นั้นอยู่จริงไหม หรือไฟล์นั้นไม่ใช่ไฟล์ นามสกุล .xml ให้ข้าม loop นั้น
    if not os.path.isfile(file_path) or not file.lower().endswith('.xml'):
        continue

    try:
        # แปลงไฟล์ นามสกุลลงท้าย .xml เป็น .png ชื่อไฟล์ xml กับ ชื่อรูปภาพเหมือนกันต่างกันแค่คนหล่ะนามสกุลไฟล์
        image_name = file.replace(".xml", ".png")
        # เอา path ที่เก็บรูปภาพมา join กับ ชื่อไฟล์รูปภาพที่พึ่งแก้นามสกุลไฟลืไป ได้เป็น path ของรูปภาพที่แท้จริง
        image_path = os.path.join(folder_path, image_name)

        # เช็คเงื่อนไขถ้า path ของรูปภาพไม่มีอยู่ ให้ข้าม loop นั้นไป
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        # เปิดรูปภาพตามตำแหน่งของ path รูปภาพแล้วแปลงรูปภาพนั้นให้เป็น grayscale
        img = Image.open(image_path).convert('L')
        # ปรับขนาดของรูปภาพที่ size หลายขนาดให้เป็นขนาด 512 x 512 (height = width)
        img = img.resize((512, 512))
        # เพิ่ม element เข้าไปใน list โดยให้รูปภาพปรับขนาดรูปร่างความกว้างและความสูงเป็น 512 และ มีช่องสีแค่ 1
        x.append(np.array(img).reshape(512, 512, 1))

        # เรียกใช้ function ได้เป็น list ที่เก็บ dictionary ที่เป็นคุณสมบัติของรูปภาพ
        objects = parse_xml(file_path)

        # เช็คถ้า objects นั้นไม่ใช้ list ว่าง (มี elements อยู่ใน list) ให้ทำเงื่อนไขด้านล่างนี้
        if objects:
            # อ่านคำตอบมาแค่ 1 คนจากรูปภาพนั้น
            # ให้ตัวแปร y เก็บ class ของคำตอบแต่ล่ะรูป
            y.append(objects[0]["label"])
        else:
            print(f"Warning: No objects found in {file_path}")
            continue
    except Exception as e:
        print(f"Error processing {file}: {e}")

# เช็คถ้าข้อมูลนั้นไม่มีอยู่ให้โยน exception นี้ออกไป
if len(x) == 0:
    raise ValueError("No data found! Check your dataset paths and XML annotations.")

# encoded ตัวคำตอบ labels แปลงจาก string -> int ค่าอยู่ในช่วง 0 - 2
y = encoder.fit_transform(y)
# แปลงข้อมูล array จากชนิด unsigned int -> float และหารด้วย 255 เพิ่อแปลงเป็น grayscale
x = np.array(x).astype('float32') / 255.0

# เช็คเงื่อนไขว่ามีข้อมูลไหม
if len(x) > 1 and len(y) > 1:
  # แบ่งข้อมูล train และ test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
else:
    print("Not enough data to split, using all data for training")

In [71]:
# แสดงผลขนาดรูปร่างของข้อมูล train และ test
# จำนวนข้อมูลรูปภาพมีทั้งหมด 853 รูป แบ่งเป็น train 682 รูป และ test 171 รูป
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(682, 512, 512, 1)
(171, 512, 512, 1)
(682,)
(171,)


### Neural Netwoerk Model

In [72]:
# สร้าง object ของ model
model = Sequential()

# เพิ่มแต่ล่ะ layers ให้ model
model.add(Flatten(input_shape=(512, 512, 1)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [74]:
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [75]:
# train model 14 รอบ
model.fit(x_train, y_train, epochs=14)

Epoch 1/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.6629 - loss: 170.3936
Epoch 2/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.6272 - loss: 93.0451
Epoch 3/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.7256 - loss: 25.7891
Epoch 4/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.7086 - loss: 2.0923
Epoch 5/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.7797 - loss: 0.8448
Epoch 6/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.7810 - loss: 0.7578
Epoch 7/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.8275 - loss: 0.7740
Epoch 8/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.7928 - loss: 0.8371
Epoch 9/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.8230 - loss: 0.7546
Epoch 10/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.7731 - loss: 0.7637
Epoch 11/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8115 - loss: 0.7431
Epoch 12/14
22/22 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 0.7955 

In [77]:
# คำนวณค่า accuracy และ ค่า loss ของ model
loss, accuracy = model.evaluate(x_test, y_test)

# แสดงผลค่าออกมา
print(f"Accuracy: {accuracy}")
print(f"Loss: {loss}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - accuracy: 0.8535 - loss: 0.6182
Accuracy: 0.8771929740905762
Loss: 0.6034767031669617


In [79]:
# บันทึกตัว model เป็นไฟล์ .keras
model.save('/content/face_mask_detection_model.keras')

In [80]:
# ทำนายผลลัพธ์จากข้อมูลทดสอบ
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# คำนวณค่า precision, recall และค่าอื่นๆ
report = classification_report(y_test, y_pred_classes, target_names=classes)

# แสดงผลลัพธ์
print(report)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step
                       precision    recall  f1-score   support

mask worn incorrectly       0.00      0.00      0.00         0
            with mask       0.88      0.99      0.93       151
         without mask       0.00      0.00      0.00        20

             accuracy                           0.88       171
            macro avg       0.29      0.33      0.31       171
         weighted avg       0.78      0.88      0.83       171



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [ ]:
# โหลดตัว model
model = load_model('/content/face_mask_detection_model.keras')